In [1]:
from measure import get_database_interface
import yaml

In [2]:
import pandas as pd
from ftplib import *
from io import StringIO
import io

In [3]:
from tqdm.auto import tqdm

In [4]:
def save_df_to_ftp(df, pretty_name, verbose=False):
    # function to put the generated values directly to FTP
    ftp = FTP('ftp-private.ebi.ac.uk')

    ftp.login('indigene_ftp', password)
    ftp.cwd('/upload/ct_scans/morphometric_analysis')
    
    buffer = StringIO()
    df.to_csv(buffer, header=True, index=False)
    text = buffer.getvalue()
    bio = io.BytesIO(str.encode(text))
    
    ftp.storbinary(f'STOR {pretty_name}.txt', bio)
    
    if verbose:
        print(ftp.dir())

In [5]:
def list_ftp():
    # function to put the generated values directly to FTP
    ftp = FTP('ftp-private.ebi.ac.uk')

    ftp.login('indigene_ftp', password) # don't forget to remove the pass before the git-committing
    ftp.cwd('/upload/ct_scans/morphometric_analysis')
    
    print(ftp.dir())

In [6]:
def get_df(iface, par_name):
    # create dataframe from the database
        
    selection = iface.as_table([par_name, 'line', 'id', 'sex', 'age', 'generation'])
    df = pd.DataFrame(selection)
    df = df.loc[~df[par_name].isna()]
    df = df.explode(par_name).explode(par_name).explode(par_name)
    df = df.rename(columns={o:n for o,n in zip([par_name, 'line', 'id', 'sex', 'age', 'generation'], 
                                               ['measurement', 'line_id', 'fish_id', 'fish_sex', 'fish_age', 'fish_generation'])})
    return df

## Connect to the database and ftp

In [7]:
with open('measurement_configs/processing/medaka_mongo.yaml') as f:
    iface = get_database_interface(yaml.safe_load(f)['db'])

In [17]:
# Getting password for the FTP

import getpass
password = getpass.getpass(prompt='Password for the FTP, please: ')
try:
    list_ftp()
except error_perm:
    print('Password is incorrect! Please, re-run this cell and try again.')

Password for the FTP, please: ········
drwxrws---    2 ftp      ftp          4096 Apr 01  2021 backup
-rw-rw----    1 ftp      ftp         18392 Aug 24 15:20 eyes.iris.color_average.txt
-rw-rw----    1 ftp      ftp         18339 Aug 24 15:20 eyes.iris.color_std.txt
-rw-rw----    1 ftp      ftp          8940 Aug 24 15:20 eyes.iris.distance_between_centers.txt
-rw-rw----    1 ftp      ftp         14371 Aug 24 15:20 eyes.iris.eccentricity_meridional.txt
-rw-rw----    1 ftp      ftp         12353 Aug 24 15:20 eyes.iris.surface_area.txt
-rw-rw----    1 ftp      ftp         12356 Aug 24 15:20 eyes.iris.volume.txt
-rw-rw----    1 ftp      ftp         18849 Aug 24 15:20 eyes.lens.color_average.txt
-rw-rw----    1 ftp      ftp         18635 Aug 24 15:20 eyes.lens.color_std.txt
-rw-rw----    1 ftp      ftp          9444 Aug 24 15:20 eyes.lens.distance_between_centers.txt
-rw-rw----    1 ftp      ftp        110576 Aug 24 15:20 eyes.lens.radius_axial.txt
-rw-rw----    1 ftp      ftp         12559 

## Adding meta-information to the database

In [136]:
scan_201811 = pd.read_excel('~/samplelist_complete.xlsx', sheet_name=4)
scan_201905 = pd.read_excel('~/samplelist_complete.xlsx', sheet_name=3)
scan_201912 = pd.read_excel('~/samplelist_complete.xlsx', sheet_name=2)
scan_202012 = pd.read_excel('~/samplelist_complete.xlsx', sheet_name=1)

all_scans = pd.concat([scan_201811, scan_201905, scan_201912, scan_202012])
all_scans = all_scans[all_scans['Scanned stained'].isin(['yes', 'Yes', 'Scanned', 'scanned'])]
all_scans = all_scans[['S.No', 'Line', 'Gen.', 'Sex', 'DOB', 'DOS']]
all_scans['age'] = (all_scans['DOS'] - pd.to_datetime(all_scans['DOB'], errors='coerce')).dt.days
all_scans = all_scans.drop(['DOS', 'DOB'], axis=1)

all_scans = all_scans.rename(columns={'S.No': 'id', 'Line': 'line', 'Gen.': 'generation', 'Sex': 'sex', 'age':'age'})
all_scans['id'] = pd.to_numeric(all_scans['id'], errors='coerce')
all_scans = all_scans[~all_scans.isna()['id']]
all_scans['id'] = all_scans['id'].astype(int).astype(str)

In [144]:
set(select_fields(db.all(), 'id')['id']) - set(all_scans.id)

set_of_reliable_ids = set(select_fields(db.all(), 'id')['id']) & set(all_scans.id)

In [145]:
for id in tqdm(set_of_reliable_ids):
    meta = all_scans[all_scans['id'] == id].iloc[0].to_dict()
    db_record = db.search(Query().id == id)[0]
    db_record.update(meta)
    db.update(db_record, Query().id == id)

## Generating the dataframes with all measurements available to be saved

In [10]:
possible_fields = set([i for rec in iface.as_table([]) for i in rec.keys()]) - {'line', 'id', 'sex', 'age', 'generation'}
possible_fields

{'eyes.iris.color_average',
 'eyes.iris.color_std',
 'eyes.iris.distance_between_centers',
 'eyes.iris.eccentricity_meridional',
 'eyes.iris.surface_area',
 'eyes.iris.volume',
 'eyes.lens.color_average',
 'eyes.lens.color_std',
 'eyes.lens.distance_between_centers',
 'eyes.lens.radius_axial',
 'eyes.lens.surface_area',
 'eyes.lens.thickness_axial',
 'eyes.lens.volume',
 'eyes.muscles.color_average',
 'eyes.muscles.color_std',
 'eyes.muscles.distance_between_centers',
 'eyes.muscles.eccentricity_meridional',
 'eyes.muscles.surface_area',
 'eyes.muscles.volume',
 'eyes.nerve.color_average',
 'eyes.nerve.color_std',
 'eyes.nerve.surface_area',
 'eyes.nerve.volume',
 'eyes.retina.color_average',
 'eyes.retina.color_std',
 'eyes.retina.distance_between_centers',
 'eyes.retina.eccentricity_meridional',
 'eyes.retina.surface_area',
 'eyes.retina.volume'}

In [150]:
## generating and uploading all datasets

for measurement_key in tqdm(possible_fields):
    df = get_df(iface, measurement_key)
    save_df_to_ftp(df, measurement_key)

In [11]:
## generating and uploading all datasets

for measurement_key in tqdm(possible_fields):
    df = get_df(iface, measurement_key)
    print(measurement_key, df.dropna(how='any').shape)

eyes.lens.volume (390, 2)
eyes.muscles.color_std (376, 2)
eyes.nerve.color_average (138, 2)
eyes.iris.color_average (361, 2)
eyes.iris.color_std (361, 2)
eyes.lens.distance_between_centers (195, 2)
eyes.lens.color_std (390, 2)
eyes.nerve.volume (195, 2)
eyes.retina.volume (390, 2)
eyes.iris.distance_between_centers (172, 2)
eyes.muscles.volume (390, 2)
eyes.iris.surface_area (390, 2)
eyes.retina.color_average (386, 2)
eyes.iris.eccentricity_meridional (290, 2)
eyes.lens.color_average (390, 2)
eyes.lens.surface_area (390, 2)
eyes.lens.thickness_axial (2064, 2)
eyes.retina.distance_between_centers (191, 2)
eyes.retina.surface_area (390, 2)
eyes.retina.eccentricity_meridional (382, 2)
eyes.muscles.color_average (376, 2)
eyes.iris.volume (390, 2)
eyes.nerve.color_std (138, 2)
eyes.lens.radius_axial (2280, 2)
eyes.muscles.eccentricity_meridional (266, 2)
eyes.retina.color_std (386, 2)
eyes.muscles.distance_between_centers (187, 2)
eyes.muscles.surface_area (390, 2)
eyes.nerve.surface_area (